In [1]:
import tensorflow as tf
from tensorflow import keras

import utils

In [2]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = utils.load_housing_data()

In [3]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [4]:
utils.reset_session()

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='selu', kernel_initializer='lecun_normal', input_shape=x_train.shape[1:]),
    keras.layers.Dense(1, activation=my_softplus,
                       kernel_regularizer=my_l1_regularizer,
                       kernel_constraint=my_positive_weights,
                       kernel_initializer=my_glorot_initializer),
])

model.compile(loss='mse', optimizer='adam', metrics=['mae'])

history = model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
363/363 [==============================] - 0s 873us/step - loss: 1.5509 - mae: 0.8976 - val_loss: 1.3818 - val_mae: 0.5622
Epoch 2/5
363/363 [==============================] - 0s 645us/step - loss: 0.5944 - mae: 0.5264 - val_loss: 1.4082 - val_mae: 0.5134
Epoch 3/5
363/363 [==============================] - 0s 634us/step - loss: 0.5258 - mae: 0.4970 - val_loss: 1.0196 - val_mae: 0.4877
Epoch 4/5
363/363 [==============================] - 0s 653us/step - loss: 0.5008 - mae: 0.4881 - val_loss: 0.8653 - val_mae: 0.4827
Epoch 5/5
363/363 [==============================] - 0s 642us/step - loss: 0.4901 - mae: 0.4828 - val_loss: 0.7505 - val_mae: 0.4788


In [5]:
model.evaluate(x_val, y_val)

121/121 [==============================] - 0s 479us/step - loss: 0.7505 - mae: 0.4788


[0.7505106329917908, 0.47877585887908936]

In [6]:
model.save('model_custom.h5')

In [7]:
utils.reset_session()

model = keras.models.load_model(
    'model_custom.h5', 
    custom_objects={
        'my_l1_regularizer': my_l1_regularizer,
        'my_positive_weights': my_positive_weights,
        'my_glorot_initializer': my_glorot_initializer,
        'my_softplus': my_softplus,
    }
)

model.evaluate(x_val, y_val)

121/121 [==============================] - 0s 372us/step - loss: 0.7505 - mean_absolute_error: 0.4788


[0.7505106329917908, 0.47877585887908936]